In [3]:
import numpy as np
from scipy.integrate import odeint

def bvpfunc(y, x, k, eps):
    phi, dphi_dx = y
    d2phi_dx2 = (k * x**2 - eps) * phi
    return [dphi_dx, d2phi_dx2]

tol = 1e-4
k = 1
L = 4
dx = 0.1
xspan = np.arange(-L, L + dx, dx)

eig_val = []
eig_vec = []

eps_start = 0
for mode in range(1, 6):
    eps = eps_start
    deps = 0.1
    
    for i in range(1000):
        y0 = [1, np.sqrt(16 - eps)]
        sol = odeint(bvpfunc, y0, xspan, args=(k, eps))
        y = sol
        
        if abs(y[-1, 1] + np.sqrt(L**2 - eps) * y[-1, 0]) < tol:
            eig_val.append(eps)
            y_norm = y[:, 0] / np.sqrt(np.trapz(y[:, 0]**2, xspan))
            eig_vec.append(np.abs(y_norm))
            break
            
        if (-1)**(mode + 1) * (y[-1, 1] + np.sqrt(L**2 - eps) * y[-1, 0]) > 0:
            eps += deps
        else:
            eps -= deps / 2
            deps /= 2
    
    eps_start = eps + 0.1

A1 = np.column_stack(eig_vec)
A2 = np.array(eig_val)

print("A1:")
print(A1)

print("A2:")
print(A2)

A1:
[[2.56183470e-04 1.45461736e-03 5.66711043e-03 1.74588478e-02
  4.50516634e-02]
 [3.76940965e-04 2.08377489e-03 7.88695717e-03 2.35399844e-02
  5.86293757e-02]
 [5.51714864e-04 2.96901781e-03 1.09156754e-02 3.15583009e-02
  7.58484043e-02]
 [8.01216066e-04 4.19588715e-03 1.49786781e-02 4.19281013e-02
  9.71899167e-02]
 [1.15311660e-03 5.87359613e-03 2.03472413e-02 5.51048300e-02
  1.23080911e-01]
 [1.64382706e-03 8.13893182e-03 2.73391697e-02 7.15653865e-02
  1.53831247e-01]
 [2.32055909e-03 1.11600069e-02 3.63163649e-02 9.17795064e-02
  1.89559870e-01]
 [3.24364379e-03 1.51394491e-02 4.76780810e-02 1.16171083e-01
  2.30113256e-01]
 [4.48904553e-03 2.03165046e-02 6.18486349e-02 1.45069155e-01
  2.74981570e-01]
 [6.15097107e-03 2.69674234e-02 7.92584569e-02 1.78649429e-01
  3.23220561e-01]
 [8.34442598e-03 3.54034079e-02 1.00317630e-01 2.16868662e-01
  3.73389602e-01]
 [1.12075190e-02 4.59653552e-02 1.25381539e-01 2.59395937e-01
  4.23518000e-01]
 [1.49032591e-02 5.90146385e-02 1.54

In [5]:
import numpy as np
from scipy.sparse.linalg import eigs

L = 4
xspan = np.arange(-L, L + 0.1, 0.1)
n = len(xspan)
dx = xspan[1] - xspan[0]

A = np.zeros((n-2, n-2))

for j in range(n-2):
    A[j, j] = -2 - (dx * xspan[j+1])**2
    if j > 0:
        A[j, j-1] = 1
    if j < n-3:
        A[j, j+1] = 1

A[0, 0] += 4/3
A[0, 1] += -1/3
A[n-3, n-4] += 4/3
A[n-3, n-3] += -1/3

eigvals, eigvecs = eigs(-A, k=5, which='SM')

v2 = np.vstack([eigvecs])
A3 = np.zeros((n, 5))

for i in range(5):
    left_adjust = (4 / 3) * eigvecs[0, i] - (1 / 3) * eigvecs[1, i]
    right_adjust = (4 / 3) * eigvecs[-1, i] - (1 / 3) * eigvecs[-2, i]
    vec = np.concatenate([[left_adjust], eigvecs[:, i], [right_adjust]])
    
    norm = np.sqrt(np.trapz(np.abs(vec)**2, xspan))
    vec /= norm
    
    A3[:, i] = np.abs(vec)

A4 = eigvals[:5] / dx**2

print("A3:")
print(A3)
print("A4:")
print(A4)

A3:
[[5.25329953e-04 2.98448351e-03 1.16780605e-02 3.63023846e-02
  9.44452586e-02]
 [5.65511305e-04 3.17858676e-03 1.23068690e-02 3.78605976e-02
  9.75009009e-02]
 [6.86055360e-04 3.76089652e-03 1.41932943e-02 4.25352367e-02
  1.06667828e-01]
 [8.98809558e-04 4.77357242e-03 1.74208187e-02 5.03835650e-02
  1.21699273e-01]
 [1.22562833e-03 6.29669498e-03 2.21637489e-02 6.16132943e-02
  1.42508910e-01]
 [1.69903993e-03 8.44716857e-03 2.86728685e-02 7.65283105e-02
  1.69044450e-01]
 [2.36360418e-03 1.13792738e-02 3.72633004e-02 9.54773771e-02
  2.01171855e-01]
 [3.27777985e-03 1.52858067e-02 4.83014917e-02 1.18800566e-01
  2.38565792e-01]
 [4.51614851e-03 2.03988758e-02 6.21889016e-02 1.46770422e-01
  2.80606814e-01]
 [6.17183762e-03 2.69894720e-02 7.93404956e-02 1.79526928e-01
  3.26289905e-01]
 [8.35896065e-03 3.53649305e-02 1.00156685e-01 2.17007357e-01
  3.74152373e-01]
 [1.12148530e-02 4.58634092e-02 1.24987977e-01 2.58874195e-01
  4.22231572e-01]
 [1.49018362e-02 5.88445566e-02 1.54

In [103]:
import numpy as np
from scipy.integrate import odeint, trapezoid

L = 2
gamma = 0.05
dx = 0.1
xspan = np.arange(-L, L + 0.1, 0.1)
A = 0.1
eps_start = 0
k = 1
tol = 1e-4

eig_val = []
eig_vec = []

def bvpfunc2(y, x, k, gamma, eps):
    dydx = np.zeros(2)
    dydx[0] = y[1]
    dydx[1] = (gamma * abs(y[0])**2 + k * x**2 - eps) * y[0]
    return dydx

for modes in range(1, 3):
    eps = eps_start
    deps = 0.1

    for i in range(1000):
        y0 = [A, A * np.sqrt(k * L**2 - eps)]

        y = odeint(bvpfunc2, y0, xspan, args=(k, gamma, eps))
        y1 = y[:, 0]
        y2 = y[:, 1]

        ynorm = trapezoid(y1**2, xspan)

        if abs(ynorm - 1) < tol:
            break
        else:
            A = A / np.sqrt(ynorm)

        if abs(y[-1, 1] + np.sqrt(L**2 - eps) * y[-1, 0]) < tol:
            break

        if (-1)**(modes + 1) * (y[-1, 1] + np.sqrt(L**2 - eps) * y[-1, 0]) > 0:
            eps += deps
        else:
            eps -= deps / 2
            deps /= 2

    eig_val.append(eps)
    eig_vec.append(y1)
    eps_start = eps + 0.1

A5 = np.column_stack(eig_vec)
A6 = np.array(eig_val)

print("A5:")
print(A5)

print("A6:")
print(A6)

A5:
[[ 1.10804361e-01  3.43447805e-01]
 [ 1.31626515e-01  3.80785689e-01]
 [ 1.55862354e-01  4.20752170e-01]
 [ 1.83563292e-01  4.62074447e-01]
 [ 2.14703052e-01  5.03279812e-01]
 [ 2.49157798e-01  5.42711445e-01]
 [ 2.86689637e-01  5.78564870e-01]
 [ 3.26934612e-01  6.08944223e-01]
 [ 3.69396510e-01  6.31936421e-01]
 [ 4.13447328e-01  6.45699751e-01]
 [ 4.58335307e-01  6.48562031e-01]
 [ 5.03200748e-01  6.39122331e-01]
 [ 5.47099458e-01  6.16349310e-01]
 [ 5.89033053e-01  5.79668653e-01]
 [ 6.27984845e-01  5.29032172e-01]
 [ 6.62959508e-01  4.64961681e-01]
 [ 6.93024365e-01  3.88562047e-01]
 [ 7.17349908e-01  3.01499719e-01]
 [ 7.35247103e-01  2.05945568e-01]
 [ 7.46199180e-01  1.04483872e-01]
 [ 7.49885914e-01 -7.64081986e-06]
 [ 7.46198835e-01 -1.04498934e-01]
 [ 7.35246416e-01 -2.05959972e-01]
 [ 7.17348885e-01 -3.01513057e-01]
 [ 6.93023013e-01 -3.88573944e-01]
 [ 6.62957835e-01 -4.64971811e-01]
 [ 6.27982860e-01 -5.29040269e-01]
 [ 5.89030768e-01 -5.79674513e-01]
 [ 5.47096884e-0

In [9]:
import numpy as np
from scipy.integrate import odeint, trapezoid

L = 2
gamma = -0.05
dx = 0.1
xspan = np.arange(-L, L + 0.1, 0.1)
A = 0.1
eps_start = 0
k = 1
tol = 1e-4

eig_val = []
eig_vec = []

def bvpfunc2(y, x, k, gamma, eps):
    dydx = np.zeros(2)
    dydx[0] = y[1]
    dydx[1] = (gamma * abs(y[0])**2 + k * x**2 - eps) * y[0]
    return dydx

for modes in range(1, 3):
    eps = eps_start
    deps = 0.1

    for i in range(1000):
        y0 = [A, A * np.sqrt(k * L**2 - eps)]

        y = odeint(bvpfunc2, y0, xspan, args=(k, gamma, eps))
        y1 = y[:, 0]
        y2 = y[:, 1]

        ynorm = trapezoid(y1**2, xspan)

        if abs(ynorm - 1) < tol:
            break
        else:
            A = A / np.sqrt(ynorm)

        if abs(y[-1, 1] + np.sqrt(L**2 - eps) * y[-1, 0]) < tol:
            break

        if (-1)**(modes + 1) * (y[-1, 1] + np.sqrt(L**2 - eps) * y[-1, 0]) > 0:
            eps += deps
        else:
            eps -= deps / 2
            deps /= 2

    eig_val.append(eps)
    eig_vec.append(y1)
    eps_start = eps + 0.1

A7 = np.column_stack(eig_vec)
A8 = np.array(eig_val)

print("A7:")
print(A7)

print("A8:")
print(A8)

A7:
[[ 1.09141599e-01  3.41249350e-01]
 [ 1.29799236e-01  3.78847162e-01]
 [ 1.53872618e-01  4.19113902e-01]
 [ 1.81424377e-01  4.60777132e-01]
 [ 2.12440950e-01  5.02357526e-01]
 [ 2.46812255e-01  5.42184307e-01]
 [ 2.84314447e-01  5.78432320e-01]
 [ 3.24596865e-01  6.09180133e-01]
 [ 3.67174551e-01  6.32487341e-01]
 [ 4.11427257e-01  6.46487311e-01]
 [ 4.56606001e-01  6.49489902e-01]
 [ 5.01847584e-01  6.40087223e-01]
 [ 5.46197123e-01  6.17254419e-01]
 [ 5.88638070e-01  5.80437086e-01]
 [ 6.28128605e-01  5.29617432e-01]
 [ 6.63642703e-01  4.65352590e-01]
 [ 6.94213682e-01  3.88780554e-01]
 [ 7.18977651e-01  3.01591646e-01]
 [ 7.37214118e-01  2.05966240e-01]
 [ 7.48381011e-01  1.04482001e-01]
 [ 7.52141700e-01 -3.77859097e-06]
 [ 7.48382022e-01 -1.04489450e-01]
 [ 7.37216123e-01 -2.05973367e-01]
 [ 7.18980631e-01 -3.01598249e-01]
 [ 6.94217604e-01 -3.88786446e-01]
 [ 6.63647532e-01 -4.65357612e-01]
 [ 6.28134301e-01 -5.29621451e-01]
 [ 5.88644594e-01 -5.80440001e-01]
 [ 5.46204438e-0

In [11]:
import numpy as np
from scipy.integrate import solve_ivp
from numpy import polyfit

def hw1_rhs_a(x, y, epsilon):
    return [y[1], (x**2 - epsilon) * y[0]]

L = 2
x_span = [-L, L]
E = 1
y0 = [1, np.sqrt(16 - E)]
tols = [1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]
solvers = ['RK45', 'RK23', 'Radau', 'BDF']
avg_step_sizes = {solver: [] for solver in solvers}

for solver in solvers:
    for tol in tols:
        options = {'rtol': tol, 'atol': tol}
        sol = solve_ivp(hw1_rhs_a, x_span, y0, method=solver, args=(E,), **options)
        avg_step_sizes[solver].append(np.mean(np.diff(sol.t)))

slopes = []
for solver in solvers:
    dt_solver = np.array(avg_step_sizes[solver])
    
    fit = np.polyfit(np.log(dt_solver), np.log(tols), 1)
    slopes.append(fit[0])

A9 = np.array(slopes)
print("A9:")
print(A9)

A9:
[5.19614684 3.02615569 4.07105005 6.58841035]


In [89]:
import numpy as np
from scipy.integrate import solve_ivp, simpson
import math
from scipy.special import hermite

def bvpfunc(x, y, k, eps):
    phi, dphi_dx = y
    d2phi_dx2 = (k * x**2 - eps) * phi
    return [dphi_dx, d2phi_dx2]

tol = 1e-4
k = 1
L = 4
A = 1
y0 = [0, A]
dx = 0.1
xspan = np.arange(-L, L + dx, dx)

eig_val = []
eig_vec = []

eps_start = 0
for mode in range(1, 6):
    eps = eps_start
    deps = 0.1
    
    for i in range(1000):
        sol = solve_ivp(bvpfunc, [xspan[0], xspan[-1]], [1, np.sqrt(16 - eps)], t_eval=xspan, args=(k, eps))
        y = sol.y.T
        temp = y[-1, 1] + np.sqrt(L**2 - eps) * y[-1, 0]
        
        if abs(temp) < tol:
            eig_val.append(eps)
            y_norm = y[:, 0] / np.sqrt(np.trapz(y[:, 0]**2, xspan))
            eig_vec.append(np.abs(y_norm))
            break
        
        if (-1)**(mode+1)*temp > 0:
            eps += deps
        else:
            eps -= deps / 2
            deps /= 2
    
    eps_start = eps + 0.1

A1 = np.column_stack(eig_vec)
A2 = np.array(eig_val)

L = 4
dx = 0.1
x = np.arange(-L, L + dx, dx)

h0 = np.ones_like(x)
h1 = 2 * x
h2 = 4 * x**2 - 2
h3 = 8 * x**3 - 12 * x
h4 = 16 * x**4 - 48 * x**2 + 12

h = np.array([h0, h1, h2, h3, h4]).T
phi = np.zeros((len(x), 5))


for j in range(5):
    phi[:, j] = (np.exp(-x**2 / 2) * h[:, j]) / (np.sqrt(2**j * math.factorial(j) * np.sqrt(np.pi))).T

eps_a = np.zeros(5)
eps_b = np.zeros(5)
er_a = np.zeros(5)
er_b = np.zeros(5)

for j in range(5):
    eps_a[j] = simpson((np.abs(A1[:, j]) - np.abs(phi[:, j]))**2, x=x)
    eps_b[j] = simpson((np.abs(A3[:, j]) - np.abs(phi[:, j]))**2, x=x)

    er_a[j] = 100 * np.abs(A2[j] - (2 * j - 1)) / (2 * j - 1)
    er_b[j] = 100 * np.abs(A4[j] - (2 * j - 1)) / (2 * j - 1)

A10 = eps_a
A11 = er_a
A12 = eps_b
A13 = er_b

print("A10:")
print(A10)
print("A11:")
print(A11)
print("A12:")
print(A12)
print("A13:")
print(A13)

A10:
[4.58285507e-08 1.73793484e-07 2.48140022e-07 4.31655500e-07
 1.92960716e-06]
A11:
[-199.97357911  199.90026019   66.61543516   39.95162003   28.51791785]
A12:
[2.73200297e-07 3.57742833e-06 3.58115990e-05 3.30999666e-04
 2.52563319e-03]
A13:
[-199.93728297  199.68205156   66.37263354   39.57364619   27.74391098]
